**Realm/Jupyter Integration Example**

This sample shows how Realm can be integrated into Kotlin DataFrames and Jupyter.

Use line magic `%use realm` to import Realm. It will automatically import the Dataframe extension library as well.

It is possible to configure both the API and Realm version using `%use realm(v=1.0.0, realm=1.7.0)`

In [1]:
%useLatestDescriptors
%use dataframe
%use lets-plot
%use @./realm.json

In [2]:
// Print information about the extension
Realm.getInfo()

Realm Kotlin Jupyter/DataFrame Extension: v0.1.0 with Realm v1.7.0-dynamic-SNAPSHOT

**1. Convert a local Realm file to Dataframes**

Reading the entire Realm file will create an artifical top-level dataframe that contains a list of all tables with their data in individual dataframes.

An encryption key can also be provided if the Realm is encrypted.

In [3]:
val df = DataFrame.readRealm("random_games.realm")
df

DataFrame: rowsCount = 3, columnsCount = 2

It is also possible to read out a single model class.

In [4]:
val df = DataFrame.readRealm(realmFile = "random_games.realm", className = "GamePlayer")
df

DataFrame: rowsCount = 2, columnsCount = 2

**2. We can also download the data from the Cloud**

Instead of using a local Realm, we can also use data from a shared database on Atlas.

In [5]:
val app = App.create("realm-jupyter-example-gszpo")
val realm: DynamicRealm = kotlinx.coroutines.runBlocking {
    val user = app.login(Credentials.anonymous())
    val config = SyncConfiguration.Builder(user = user, partitionValue = "example", schema = setOf())
        .waitForInitialRemoteData()
        .build()
    DynamicRealm.open(config)
}

// Convert the entire Realm to a dataframe
val df = realm.toDataFrame()
df

DataFrame: rowsCount = 3, columnsCount = 2

**2. We can also convert RealmResults, RealmList, RealmSet and RealmObject directly to a dataframe**

Instead of using `DataFrame` as the entry point, we can use a open `Realm` and convert different parts of the open Realm into a dataframe.

Converting query results to dataframes.

In [6]:
val results: RealmResults<out DynamicRealmObject> = realm.query("GamePlayer").sort("name").find()
results.toDataFrame(realm)

DataFrame: rowsCount = 2, columnsCount = 4

Converting single objects

In [7]:
val obj: DynamicRealmObject = results.first()
obj.toDataFrame(realm)

DataFrame: rowsCount = 1, columnsCount = 4

**3. Links are exposed as column groups**

Links are exposed as column groups which gives a nicer visual representation than exposing them as dataframes. Either way, it is straightforward to manipulate the structure using either `explode`, `flatten` or `ungroup`.

In [8]:
val df = DataFrame.readRealm(realmFile = "random_games.realm", className = "SaveGame")
df

... showing only top 20 of 1000 rows DataFrame: rowsCount = 1000, columnsCount = 4

**4. Work with the data**

Once the Realm has been converted into a dataframe, all the data has been copied into memory and be transformed similar to any other dataframe.

In this example, lets clean it up and calculate some statistics about Tic Tac Toe.

In [9]:
// Find the steps leading up to a specific player winning. `null` means a draw.
val gameMoves = df.remove { player1 and player2 }.flatten { winner }.remove("name").rename("type" to "winner")
gameMoves

... showing only top 20 of 1000 rows DataFrame: rowsCount = 1000, columnsCount = 2

In [10]:
val gameResults = gameMoves.groupBy { winner }.count()
gameResults

DataFrame: rowsCount = 3, columnsCount = 2

In [11]:
// Process the data so it is usable in Lets-plot.package
// - Rename `null` to draw`
// - Add a "percentage" column used when rendering the bar plot.
val totalGames = gameResults.count.sum()
val barData = gameResults
    .update("winner").where { this["winner"] == null }.with { "draw" }
    .add("percentage") { count / totalGames.toFloat() }
barData

DataFrame: rowsCount = 3, columnsCount = 3

In [12]:
// Plot the data
val plotSize = ggsize(400,250)
val blankTheme = theme(line=elementBlank(), axis=elementBlank())
val tooltipContent = layerTooltips()
                        .line("games|@count (@percentage)")
                        .line("total|${totalGames}")
                        .format("percentage", ".01%")

letsPlot(barData.toMap()) + plotSize +
    geomBar(
        tooltips = tooltipContent,
    ) { x = "winner"; y = "count"; fill = "winner"; weight = "count" }

Now calculate what the odds of X winning are based on the first move.

In [ ]:
val firstMoves = gameMoves.add("firstMove") { steps[0] }.remove { steps }.flatten()
firstMoves

In [ ]:
fun getHeatMapForOutcome(token: String): AnyFrame {
    return firstMoves.filter { winner == token }
        .groupBy { x and y }.count()
        .add("percentage") { 
            // Count is added as a calculated field by the groupBy operation
            (this["count"] as Int) / this.df().sum("count").toFloat()
        }
}

val heatmapX = getHeatMapForOutcome("X")
heatmapX

In [ ]:
letsPlot(heatmapX.toMap()) + plotSize +
    geomTile() { x = "x"; y = "y"; fill = "percentage"; } + 
    geomText(color = "white", labelFormat = ".1%") { x = "x"; y = "y"; label = "percentage"} 

Finally, any resulting dataframes can be written back into the Realm as long as the schema matches.

In [ ]:
heatmapX.copyToRealm(realm, "GameStatistics")